In [5]:
dtable = [1, 1, 2, 3, 5, 1, 6, 0, 
          5, 5, 3, 1, 4, 5, 2, 0, 
          4, 4, 1, 5, 6, 4, 3, 0, 
          6, 6, 5, 4, 2, 6, 1, 0, 
          2, 2, 4, 6, 3, 2, 5, 0, 
          3, 3, 6, 2, 1, 3, 4, 0]

In [321]:
class Fighter:
    def __init__(self, name, hp, armor, moves):
        self.name = name
        self.hp = hp
        self.armor = armor
        self.moves = moves
        
class Move:
    def __init__(self, movename, dice_req, rolls, en_cost=0, atk_val=0, en_gain=0, max_en=999):
        self.movename = movename
        self.dice_req = dice_req
        self.en_cost = en_cost
        self.rolls = rolls
        self.atk_val = atk_val
        self.en_gain = en_gain
        self.max_en = max_en
        


In [335]:
m_charge = Move(movename = 'Charge', dice_req = [0,1,2,3,4,5,6], rolls=1, en_gain=1)

f_rock = Fighter(name='Rock', hp=8, armor=1, moves=[
    m_charge,
    Move(movename='Atk1', dice_req = [2, 4], en_cost = 1, atk_val = 2, rolls = 2),
    Move(movename='Atk2', dice_req = [3, 5], en_cost = 3, atk_val = 3, rolls = 3)
])

f_paper = Fighter(name='Paper', hp=9, armor=0, moves=[
    m_charge,
    Move(movename='ChargePlus', dice_req = [5], rolls = 4, en_gain = 2),
    Move(movename='Atk', dice_req = [1, 4], en_cost = 5, atk_val = 5, rolls = 3)
])

f_scissors = Fighter(name='Scissors', hp=7, armor=1, moves=[
    m_charge,
    Move(movename='Bite', dice_req = [1,2,3,4], atk_val = 1, rolls = 2),
    Move(movename='Snip', dice_req = [3,4], en_cost = 3, atk_val = 3, rolls = 3)
])


In [336]:
import numpy as np
from numpy.random import choice

In [337]:
class Player:
    def __init__(self, player_no, hp, en, current_fighter):
        self.player_no = player_no
        self.hp = hp
        self.en = en
        self.current_fighter = current_fighter
    
    def modify_hp(self, delta):
        return Player(player_no=self.player_no, hp=self.hp+delta, en=self.en, current_fighter=self.current_fighter)
    
    def modify_en(self, delta):
        return Player(player_no=self.player_no, hp=self.hp, en=self.en+delta, current_fighter=self.current_fighter)
    
    def __str__(self):
        return 'Player ' + str(self.player_no) + ' -  Fighter: ' + self.current_fighter.name + ' - HP: ' + str(self.hp) + ' - EN: ' + str(self.en)
        
class GameState:
    def __init__(self, whose_turn, players, dice_loc, 
                 parent_state = None, child_states = None, child_states_computed = False, value=None):
        self.whose_turn = whose_turn
        self.players = players
        self.dice_loc = dice_loc
        self.parent_state = parent_state
        self.value = value
        self.child_states = child_states
        self.child_states_computed = child_states_computed
    
    def get_dice(self):
        return dtable[self.dice_loc]
        
    def new_game(f1, f2, dice_loc):
        p1 = Player(player_no=1, hp=f1.hp, en=0, current_fighter=f1)
        p2 = Player(player_no=2, hp=f2.hp, en=0, current_fighter=f2)
        return GameState(whose_turn=1, players = {1:p1, 2:p2}, dice_loc=dice_loc)
    
    def compute_child_states(self):
        if self.child_states_computed:
            return
        dice = self.get_dice()
        nturn = 3-self.whose_turn
        cp = self.players[self.whose_turn]
        np = self.players[nturn]
        self.child_states = []
        if cp.hp <= 0:
            return
        cf = cp.current_fighter
        nf = np.current_fighter
        for m in cf.moves:
            if (dice in m.dice_req) and (cp.en >= m.en_cost) and (cp.en <= m.max_en):
                cp_mod = cp.modify_en(m.en_gain - m.en_cost)
                newloc = (self.dice_loc + m.rolls) % len(dtable)
                atk_dmg = max(0, m.atk_val - nf.armor)
                np_mod = np.modify_hp(max(-np.hp, -atk_dmg))
                new_players={cp_mod.player_no:cp_mod,
                             np_mod.player_no:np_mod}
                new_state = GameState(whose_turn=nturn, players=new_players, dice_loc=newloc, parent_state=self)
                self.child_states.append(new_state)
        self.child_states_computed = True
        return
    
    def __str__(self):
        return 'Loc: ' + str(self.dice_loc) + ' (dice ' + str(self.get_dice()) + ')\n' + \
        str(self.players[1]) + '\n' + \
        str(self.players[2]) + '\nPlayer ' + str(self.whose_turn) + ' turn'
        
    
    def compute_value(self, iter_limit = 10):
        if (iter_limit == 0):
            return 0
        if (self.value is not None):
            return self.value
        if (self.players[1].hp <= 0) and (self.players[2].hp > 0):
            self.value = -1
            return self.value
        if (self.players[1].hp > 0) and (self.players[2].hp <= 0):
            self.value = 1
            return self.value
        if (self.players[1].hp <= 0) and (self.players[2].hp <= 0):
            self.value = 0
            return self.value
        self.compute_child_states()
        assert(len(self.child_states)) > 0 # if empty, it must be a terminal state and hence would have returned already
        child_values = []
        for c in self.child_states:
            child_values.append(c.compute_value(iter_limit - 1))
        if self.whose_turn==1:
            self.value = max(child_values)
            return self.value
        if self.whose_turn==2:
            self.value = min(child_values)
            return self.value
    
    def n_descendents(self):
        if (self.child_states is None):
            return 0
        if len(self.child_states) == 0:
            return 0
        return sum([c.n_descendents() for c in self.child_states]) + 1
    
    def random_descendent(self):
        if (self.child_states is None):
            return self
        if len(self.child_states) == 0:
            return self
        return choice(self.child_states).random_descendent()

In [338]:
gs = GameState.new_game(f_scissors, f_rock, 8)

In [339]:
gs.compute_value(30)

0

In [340]:
print(gs.random_descendent())

Loc: 1 (dice 1)
Player 1 -  Fighter: Scissors - HP: 3 - EN: 4
Player 2 -  Fighter: Rock - HP: 4 - EN: 7
Player 1 turn


In [341]:
cs = gs.random_descendent()

In [342]:
cs.child_states_computed

False

In [343]:
cs.compute_value(5)

-1

In [344]:
print(cs.random_descendent())

Loc: 7 (dice 0)
Player 1 -  Fighter: Scissors - HP: 1 - EN: 10
Player 2 -  Fighter: Rock - HP: 6 - EN: 5
Player 2 turn
